# Agent Basics

This notebooks walks you through creating two agents that work together to schedule coffee.

First we create two agents: 

In [1]:
from osnap_client.agents import (
  OSNAPBaseAgent, 
  AgentInfo, 
  AgentTask, 
  AgentRunResponse,
  AgentTaskResult
)

def alice_run():
  pass

def alice_start(): 
  pass

def alice_listen(): 
  pass

def alice_complete():
  pass

def alice_terminate():
  pass

class AliceAgent(OSNAPBaseAgent):
  name="alice"
  description="Alice's personal assistant"
  tools=["calendar", "email", "location"]
  id="1234"

  def run(self, task: AgentTask):
    return alice_run(self, task)

  def start(self, objective: str, agent_url=None):
    return alice_start(self, objective, agent_url)

  def listen(self, result: AgentTaskResult):
    return alice_listen(result)
  
  def complete(self):
    return alice_complete()
  
  def terminate(self):
    return alice_terminate()
  
alice = AliceAgent()



In [48]:
# define the methods so we can test them
def bob_run():
  pass

def bob_start():
  pass

def bob_listen():
  pass

def bob_complete():
  pass

def bob_terminate():
  pass

class BobAgent(OSNAPBaseAgent):
  name="bob"
  description="Bobs's personal assistant"
  tools=["calendar", "email", "location"]
  id="1234"

  def run(self, task: AgentTask) -> AgentRunResponse:
    return bob_run(task)

  def start(self, objective: str, agent_url=None):
    return bob_start(objective, agent_url)

  def listen(self, result: AgentTaskResult):
    return bob_listen(result)
  
  def complete(self, payload: dict = {}):
    return bob_complete(payload)
  
  def terminate(self):
    return bob_terminate()
  
bob = BobAgent()

bob.info()

AgentInfo(name='bob', description="Bobs's personal assistant", id=1234, tools=['calendar', 'email', 'location'], url='')

In your application you will define the required methods when you create an object, but for the purposes of this tutorial we will define them one at a time as we go. Like the following example:

```python 
def alice_run(task: OSNAPTask):
  print(f"Alice is running task NOW!! {task}")
  
alice.run("something")
```   

First let's say that Alice wants her agent to work with Bob's agent to schedule coffee with him. 
We initialize this request by calling the `start` method on Alice's agent. In our case, we are going to hardcode some logic into Alice's agent to make it easier to understand. 


In [4]:
import openai
import getpass
openai.api_key = getpass.getpass()

bob_info = bob.info()

In [5]:
def alice_start(self, objective, agent_url=None):
    prompt = f"""
    You are {self.description}. You have access to the following tools: {self.tools}.

    Alice wants to {objective}. Bob's agent is {bob_info.description}. Bob's agent has access to the following tools: {bob_info.tools}.

    What are the tasks we need to complete {objective}?
    """

    response = openai.Completion.create(
      engine="text-davinci-003",
      prompt=prompt,
      temperature=0.5,
      max_tokens=200
    )

    return response

In [6]:
response = alice.start(objective="Schedule coffee with Bob")

In [7]:
task_list = response.choices[0].text.split("\n")

In [8]:
task_list

['',
 '1. Determine a suitable date and time for the meeting.',
 "2. Send an email to Bob's agent to confirm the date and time.",
 "3. Use the calendar to set a reminder for Alice and Bob's agent.",
 '4. Use the location tool to find a suitable location for the meeting.',
 "5. Send an email to Bob's agent to confirm the location."]

The protocol isn't opinionated about WHAT your agent does. In this case we are just picking a hardcoded external agent, and then using it's description and tools directly in the prompt. This allows every agent to define as much logic as they want. The purpose of the protocol is to make them interoperable. 

Now that we have a nice list of tasks, we can start asking Bob's agent for help

In [9]:
prompt = f"""
You are {alice.description}. You have access to the following tools: {alice.tools}.
Alice wants to Schedule coffee with Bob. Bob's agent is {bob_info.description}. Bob's agent has access to the following tools: {bob_info.tools}.

The first task is to {task_list[1]}.

We're going to make a simple request to Bob's agent to help. What is the request?
Format your request as a simple task for Bob's agent to complete. 
You can also specify a tool to use, if you want to be specific.
"""
response = openai.Completion.create(
  engine="text-davinci-003",
  prompt=prompt,
  temperature=0.5,
  max_tokens=200
)

In [10]:
task_map = {}
task_map[response.id] = response.choices[0].text

Now that we have a task ready for Bob's agent, we're going to send it to him. 
We give it an id here so that both agents can keep track of it.

Let's turn this into an OSNAPTaskRequest:

In [11]:
task1 = AgentTask(
  objective="Schedule coffee with Bob",
  task_id=response.id,
  task_name=task_map[response.id],
  task_description=task_map[response.id],
  task_tool="calendar"
)

Now we can implement the run method on Bob's agent that we just stubbed out later. 

run takes an OSNAPTask as an input and returns an OSNAPAgentRunResponse. 

Run might be a long running task, so we want to return a status that indicates to the caller if that it's working on it.
Let's just have it return a status of "starting" for now.

In [12]:
def bob_run(task): 
    print("Incoming task from Alice: ", task.task_name)
    return AgentRunResponse(
        status="starting",
        message="Sounds good, I'll get started on that.",
        payload={
            "tools": ["calendar"],
        }
    )

bob_run_response = bob.run(task1)

Incoming task from Alice:  
Request: Please use the calendar tool to find a suitable date and time for Alice and Bob to meet for coffee.


## Async task handling 

Since we mentioned that Agent Tasks can take a while, how does one agent let another know that it's task is complete? There are many different ways that distributed systems handle this, but OSNAP doesn't prescribe a particular implementation. 

All that is required is that agents who make task requests register a "listener" that the other agent can use when they have a task result. This is essntially the [Observer Pattern](https://en.wikipedia.org/wiki/Observer_pattern). 

Let's try it out!

In [40]:
alice_assigned_tasks = {}
alice_completed_tasks = {}

if bob_run_response.status == "starting" or bob_run_response.status == "working":
    alice_assigned_tasks[task1.task_id] = task1
    print("Alice has assigned the following tasks: ")
    print({task.task_name for task in alice_assigned_tasks.values()})

def alice_listen(result: AgentTaskResult):
    print("Bob's agent has completed the task: ", result.task_name)
    print("Bob's agent task result says: ", result.message)
    completed = alice_assigned_tasks.pop(result.task_id)
    alice_completed_tasks[result.task_id] = completed


Alice has assigned the following tasks: 
{'\nRequest: Please use the calendar tool to find a suitable date and time for Alice and Bob to meet for coffee.'}


We give alice a [Python set](https://docs.python.org/3/tutorial/datastructures.html#sets) of assigned tasks, but this can be implemented however you chose, like with a more sophisticated tool like Kafka or Redis PubSub for example. 

We look at the `AgentRunResponse` that Bob's agent gave us when we sent the `AgentTask` and see if we want to listen for a result by just checking if the status is `"starting"` or `"working"`. 

Then in Alice's listen method, if a task completes, we remove that task from the assigned tasks set. 

In [41]:
bob_task_result = AgentTaskResult.construct(
    task_id=task1.task_id,
    task_name=task1.task_name,
    status="success",
    message="Bob is available at 3pm on Tuesday.",
    payload={}
)

# bob log? bob loblaw's law blog?
bob_log = [
    (bob_task_result, "calendar")
]

alice.listen(bob_task_result)
print("Alice has assigned the following tasks: ")
print({task.task_name for task in alice_assigned_tasks.values()})

Bob's agent has completed the task:  
Request: Please use the calendar tool to find a suitable date and time for Alice and Bob to meet for coffee.
Bob's agent task result says:  Bob is available at 3pm on Tuesday.
Alice has assigned the following tasks: 
set()


Bob's agent does some stuff, and decides that Bob is available at 3pm on Tuesday. We will also keep track of the tasks bob did in a task log for reference later. Now Bob's agent can call Alice's agent listen method with the task result. 

Now we can see that Alice has removed that task from her listeners. Notice that if you run that cell twice, you'll get an error. This actually somewhat intentional as it makes sense that Alice has already stopped listening for that task. This isn't a part of the protocol per se, Alice's agent could keep listening for more results for a particular task (like for example if she wanted to distribute it to multiple different agents instead), it's up to you as a devloper to figure out what you want to do. 

### Next Step

Let's say for the sake of demonstration that Alice's agent checks her calendar and sees that 3pm on Tuesday works for her as well. Let's construct a prompt to help Alice's agent figure out what it wants to do next.

In [31]:
alice_schedule_response = "Tuesday at 3pm works for Alice as well"

prompt = f"""
You are {alice.description}. You have access to the following tools: {alice.tools}.

Alice wants to schedule coffee with bob. Bob's agent is {bob_info.description}. Bob's agent has access to the following tools: {bob_info.tools}.

Previously we had the following tasks: {task_list}

Bob's agent has completed the following task: {bob_task_result.task_name}. Bob's agent task result says: {bob_task_result.message}

{alice_schedule_response}

What are the remaining tasks we need to the objective of scheduling coffee with bob?
"""

response = openai.Completion.create(
  engine="text-davinci-003",
  prompt=prompt,
  temperature=0.5,
  max_tokens=200
)

In [33]:
response.choices[0].text.split("\n")

['',
 "1. Send an email to Bob's agent to confirm the date and time of 3pm on Tuesday.",
 "2. Use the calendar to set a reminder for Alice and Bob's agent.",
 '3. Use the location tool to find a suitable location for the meeting.',
 "4. Send an email to Bob's agent to confirm the location.",
 '5. Send an email to Alice and Bob to confirm the date, time, and location of the meeting.']

Groovy! The great part of autonomous agents is that they can keep running and completing tasks, sometimes without any user input. 

The agents can continue to work together using the same back and forth protocol we showed above for the rest of the tasks. 

There's just one last thing we need to do. Alice is satisfied with what her agent has done so far to achieve the `Objective` of scheduling coffee with Bob. She should let Bob's agent know that. This what the `complete` method is for. 

In [51]:
def bob_complete(payload): 
    print(f"""
        I worked with {payload['agent_name']} on the objective: {payload['objective']}.

        We completed the following tasks: {payload['tasks']}

        I used the following tools: {[tool for _, tool in bob_log]}
    """)

alice_summary = {
    "agent_name": alice.name,
    "objective": "Schedule coffee with Bob",
    "tasks": [task.task_name for task in alice_completed_tasks.values()],
}

bob.complete(payload=alice_summary)


        I worked with alice on the objective: Schedule coffee with Bob.

        We completed the following tasks: ['\nRequest: Please use the calendar tool to find a suitable date and time for Alice and Bob to meet for coffee.']

        I used the following tools: ['calendar']
    


## Wrapping Up

This is the very basic, 10,000 foot overview of how to build a very basic two agent flow. 